# EDA Crossover Strategy Data Wrangling

In [1]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import random
import requests
from bs4 import BeautifulSoup
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report, confusion_matrix
import mplfinance as mpf

In [18]:
def get_sp500_tickers():
    url = 'https://www.slickcharts.com/sp500'
    html = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}).text
    table = pd.read_html(str(BeautifulSoup(html, 'lxml').find('table')))[0]
    return table['Symbol'].str.replace('.', '-', regex=False).tolist()

tickers = get_sp500_tickers()
df = yf.download(tickers, start='2010-01-01', end='2025-04-22', group_by='ticker', auto_adjust=True)

C:\Users\rusla\AppData\Local\Temp\ipykernel_12460\1915853785.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(str(BeautifulSoup(html, 'lxml').find('table')))[0]
[*********************100%***********************]  503 of 503 completed


In [31]:
df

Price,Open,High,Low,Close,Volume,Open,High,Low,Close,Volume,...,Open,High,Low,Close,Volume,Open,High,Low,Close,Volume
Ticker,KVUE,KVUE,KVUE,KVUE,KVUE,CRWD,CRWD,CRWD,CRWD,CRWD,...,IPG,IPG,IPG,IPG,IPG,CSCO,CSCO,CSCO,CSCO,CSCO
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.703628,4.810959,4.672060,4.754137,4444300,15.873471,16.354085,15.807633,16.255329,59853700
2010-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.760452,4.760452,4.678375,4.703629,6134700,16.196077,16.281665,16.051233,16.182909,45124500
2010-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.672062,4.709943,4.621553,4.703629,5586100,16.156576,16.288250,16.024900,16.077570,35715700
2010-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.653122,4.703631,4.539477,4.583673,6202400,15.998564,16.176326,15.912976,16.149992,31531200
2010-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.589985,4.608926,4.470027,4.589985,6480800,16.051240,16.261922,15.965652,16.235586,39115900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-14,22.370001,22.889000,22.190001,22.709999,16487900.0,386.954987,389.252991,372.820007,378.660004,3048000.0,...,24.980000,25.280001,24.690001,25.010000,5308600,57.889999,58.209999,57.099998,57.410000,15657700
2025-04-15,22.709999,23.120001,22.559999,22.910000,22262800.0,383.399994,400.019989,379.670013,392.799988,5204800.0,...,25.219999,25.440001,25.040001,25.139999,4096900,57.869999,57.910000,57.060001,57.200001,18543600


In [19]:
# Multi to singlular column transformation
df.columns = df.columns.swaplevel(0, 1)
df_flat = df.stack(level=1, future_stack=True).reset_index()
df_flat.columns.name = None
df_flat.columns = [col if not isinstance(col, tuple) else col[-1] for col in df_flat.columns]
df_flat.head()

,Date,Ticker,Open,High,Low,Close,Volume
0,2010-01-04,KVUE,NaN,NaN,NaN,NaN,NaN
1,2010-01-04,CRWD,NaN,NaN,NaN,NaN,NaN
2,2010-01-04,TPR,24.332192,24.332192,23.817715,23.949633,3350600.0
3,2010-01-04,INVH,NaN,NaN,NaN,NaN,NaN
4,2010-01-04,UNH,24.976561,25.151668,24.650226,25.095953,12199500.0


In [20]:
df_flat.shape

(1935544, 7)

In [21]:
df_flat.dtypes

Date      datetime64[ns]
Ticker            object
Open             float64
High             float64
Low              float64
Close            float64
Volume           float64
dtype: object

In [22]:
df_flat.describe().T

,count,mean,min,25%,50%,75%,max,std
Date,1935544,2017-08-24 16:08:28.939709440,2010-01-04 00:00:00,2013-10-28 18:00:00,2017-08-23 12:00:00,2021-06-21 06:00:00,2025-04-21 00:00:00,NaN
Open,1833471.0,99.347589,0.199906,28.330695,53.553143,104.760002,9914.169922,226.765306
High,1833471.0,100.47082,0.207471,28.653362,54.122738,105.905588,9964.769531,229.472287
Low,1833471.0,98.204704,0.198301,28.006,52.98807,103.599998,9794.0,224.120323
Close,1833471.0,99.361487,0.203574,28.338123,53.56591,104.775452,9924.400391,226.804188
Volume,1833471.0,7010164.52829,0.0,1017500.0,2227000.0,5104800.0,3692928000.0,32735749.441142


In [23]:
df_flat.nunique()

Date         3848
Ticker        503
Open      1607079
High      1609585
Low       1608484
Close     1493398
Volume     369494
dtype: int64

In [24]:
df_flat.isnull().sum()

Date           0
Ticker         0
Open      102073
High      102073
Low       102073
Close     102073
Volume    102073
dtype: int64

In [25]:
df_flat[df_flat.isnull().any(axis=1)]

,Date,Ticker,Open,High,Low,Close,Volume
0,2010-01-04,KVUE,NaN,NaN,NaN,NaN,NaN
1,2010-01-04,CRWD,NaN,NaN,NaN,NaN,NaN
3,2010-01-04,INVH,NaN,NaN,NaN,NaN,NaN
10,2010-01-04,HLT,NaN,NaN,NaN,NaN,NaN
30,2010-01-04,PAYC,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1833893,2024-06-28,SW,NaN,NaN,NaN,NaN,NaN
1834396,2024-07-01,SW,NaN,NaN,NaN,NaN,NaN
1834899,2024-07-02,SW,NaN,NaN,NaN,NaN,NaN
1835402,2024-07-03,SW,NaN,NaN,NaN,NaN,NaN


In [26]:
df_flat.dropna(inplace=True)
df_flat.isnull().sum()

Date      0
Ticker    0
Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64

In [29]:
df_flat.shape

(1833471, 7)

In [30]:
df_flat.head()

,Date,Ticker,Open,High,Low,Close,Volume
2,2010-01-04,TPR,24.332192,24.332192,23.817715,23.949633,3350600.0
4,2010-01-04,UNH,24.976561,25.151668,24.650226,25.095953,12199500.0
5,2010-01-04,PKG,14.846490,15.139974,14.820969,14.923051,810900.0
6,2010-01-04,REG,19.598867,19.809847,19.154701,19.304607,1354400.0
7,2010-01-04,COO,9.503958,9.558607,9.342495,9.377272,1924800.0
